In [37]:
#Importing Libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import precision_score, recall_score, f1_score

In [38]:
#Importing Dataset
data = pd.read_csv("anime.csv")

In [39]:
#Data Preprocessing
print("\nData Shape : ")
print(data.shape)
print("\nStatistical Summary : ")
print(data.describe())
print("\nInitial Values : ")
print(data.head())

data.dropna(subset=['name','genre','rating'],inplace=True)
data = data.reset_index(drop=True)

scaler = MinMaxScaler()
data['rating_norm'] = scaler.fit_transform(data[['rating']])

data['features'] = (
    data['genre'] + " " +
    data['type'].astype(str) + " " +
    data['rating_norm'].astype(str)
)



Data Shape : 
(12294, 7)

Statistical Summary : 
           anime_id        rating       members
count  12294.000000  12064.000000  1.229400e+04
mean   14058.221653      6.473902  1.807134e+04
std    11455.294701      1.026746  5.482068e+04
min        1.000000      1.670000  5.000000e+00
25%     3484.250000      5.880000  2.250000e+02
50%    10260.500000      6.570000  1.550000e+03
75%    24794.500000      7.180000  9.437000e+03
max    34527.000000     10.000000  1.013917e+06

Initial Values : 
   anime_id                              name  \
0     32281                    Kimi no Na wa.   
1      5114  Fullmetal Alchemist: Brotherhood   
2     28977                          Gintama°   
3      9253                       Steins;Gate   
4      9969                     Gintama&#039;   

                                               genre   type episodes  rating  \
0               Drama, Romance, School, Supernatural  Movie        1    9.37   
1  Action, Adventure, Drama, Fantasy, Magic,

In [40]:
#Vectorization
vectorizer = TfidfVectorizer(stop_words='english')
feature_matrix = vectorizer.fit_transform(data['features'])

cosine_sim = cosine_similarity(feature_matrix,feature_matrix)

In [41]:
# Train-Test Split (80% training titles, 20% testing titles)
train_data, test_data = train_test_split(data, test_size=0.2, random_state=42)

#train_data = train_data.reset_index(drop=True)
#test_data = test_data.reset_index(drop=True)

train_indices = train_data.index
test_indices = test_data.index

print("\nTrain Size:", len(train_data))
print("Test Size:", len(test_data))



Train Size: 9613
Test Size: 2404


In [ ]:

def evaluate_recommender(k=5):
    y_true = []
    y_pred = []

    for idx in test_data.index:
        test_genres = set(test_data.loc[idx, 'genre'].split(','))

        # similarity scores for this test item
        sim_scores = list(enumerate(cosine_sim[idx]))

        # consider only training items
        sim_scores = [(i, score) for i, score in sim_scores if i in train_data.index]

        sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
        top_k_indices = [i for i, _ in sim_scores[:k]]

        # Check relevance: genre overlap
        relevant = 0
        for i in top_k_indices:
            train_genres = set(data.loc[i, 'genre'].split(','))
            if test_genres & train_genres:
                relevant = 1
                break

        y_true.append(1)
        y_pred.append(relevant)

    precision = precision_score(y_true, y_pred)
    recall = recall_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred)

    print(f"\nEvaluation Metrics (Top-{k}):")
    print("Precision:", precision)
    print("Recall:", recall)
    print("F1 Score:", f1)

evaluate_recommender(k=5)
evaluate_recommender(k=10)



print("\nInterpretation of Evaluation Metrics:\n")

print("Precision indicates how many recommended anime were actually relevant.")
print("A lower precision suggests some recommendations are not strongly related.")

print("Recall shows how well the system retrieves relevant anime from the dataset.")
print("Lower recall indicates that some relevant anime are being missed.")

print("F1-score balances precision and recall.")
print("Improving feature representation can increase all three metrics.")

print("\nImprovement Strategies:")
print("1. Incorporate numerical features such as ratings.")
print("2. Assign higher weight to genre similarity.")
print("3. Use dimensionality reduction to remove noise.")
print("4. Combine content-based filtering with collaborative signals.")



Evaluation Metrics (Top-5):
Precision: 1.0
Recall: 0.8993344425956739
F1 Score: 0.946999561979851


In [ ]:
def recommend_with_threshold(title, threshold=0.20):
    if title not in data['name'].values:
        print("Anime not in database.")
        return
    
    idx = data[data['name'] == title].index[0]
    sim_scores = list(enumerate(cosine_sim[idx]))

    # Filter by similarity threshold
    filtered = [i for i in sim_scores if i[1] >= threshold and i[0] != idx]

    if len(filtered) == 0:
        print("\nNo recommendations found above threshold.")
        return

    filtered = sorted(filtered, key=lambda x: x[1], reverse=True)

    print(f"\nRecommendations for '{title}' (Similarity ≥ {threshold}):\n")
    for i, score in filtered:
        print(f"{data.loc[i, 'name']}  -> similarity: {round(score, 3)}")


In [ ]:
def recommend_anime(title,num_recommendations):
    if title not in data['name'].values:
        print("Anime not in database .")
        return

    idx = data[data['name'] == title].index[0]
    sim_score = list(enumerate(cosine_sim[idx]))
    sim_score = sorted(sim_score,key=lambda x:x[1],reverse=True)
    top_indices = [i[0] for i in sim_score[1:num_recommendations+1]]

    print(f"\nRecommended Animes similar to '{title} : \n")
    print(data['name'].iloc[top_indices].to_string(index=False))


recommend_anime("Cowboy Bebop",5)

In [ ]:
"""Performance Analysis:

The recommendation system uses content-based filtering with TF-IDF vectorization
and cosine similarity.

Precision indicates that a majority of recommended anime share similar genres
with the target anime.

Recall shows that the system retrieves a reasonable portion of relevant anime,
though some relevant items may be missed.

The F1-score demonstrates a balanced trade-off between precision and recall.

Performance improves when increasing the value of K, but at the cost of
recommendation relevance.

Areas of improvement include:
1. Using genre weighting instead of equal importance.
2. Incorporating numerical ratings more effectively.
3. Hybridizing with collaborative filtering.
4. Reducing feature noise using dimensionality reduction (SVD).
"""

In [ ]:
"""1. Difference Between User-Based and Item-Based Collaborative Filtering

User-Based CF recommends items based on similar users.

It finds users with similar tastes using similarity measures (cosine, Pearson).

Example: “People who watch the same movies as you also liked X.”

It works well when user behavior patterns are clear.

But it becomes slow when there are many users.

Item-Based CF recommends items similar to what the user liked.

Similarity is calculated between items instead of users.

Example: “People who liked Movie A also liked Movie B.”

Item-based is faster and more stable because items don’t change often.

Modern systems like Amazon mainly use item-based CF.

2. What is Collaborative Filtering, and How Does It Work?

Collaborative Filtering (CF) is a recommendation technique.

It makes predictions based on user behavior (ratings, purchases, clicks).

CF assumes “users with similar tastes will like similar items.”

It creates a matrix of users vs items and finds patterns.

User-based CF finds similar users to recommend items.

Item-based CF finds similar items the user may like.

It uses similarity metrics like cosine similarity or Pearson correlation.

Works well without item metadata (actors, genre, product info).

Learns from crowdsourced behavior instead of manual rules.

Used in Netflix, Amazon, Spotify, and YouTube recommendations."""